In [1]:
#Important data Collection Libraries
import os
import glob
import sys
import tensorflow as tf

from scipy import misc
import numpy as np

In [3]:
from tensorflow.contrib.keras.python import keras
from tensorflow.contrib.keras.python.keras import layers, models
from tensorflow import image

In [5]:
from utils import scoring_utils
from utils.separable_conv2d import SeparableConv2DKeras,BilinearUpSampling2D
from utils import data_iterator
from utils import plotting_tools
from utils import model_tools

### Seperable Convolutions - FCN layers

In [1]:
def separable_conv2d_batchnorm(input_layer, filters, strides=1):
    output_layer = SeparableConv2DKeras(filters=filters,kernel_size=3, strides=strides,
                             padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

def conv2d_batchnorm(input_layer, filters, kernel_size=3, strides=1):
    output_layer = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                      padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

### Bilinear Upsampling 

It is used in the decoder block of FCN. Upsampling by a factor 2. The "utils.separable_conv2d" folder contains the implementation of customized form of Keras UpSampling2D named BilinearUpSampling2D.

In [2]:
def bilinear_upsample(input_layer):
    output_layer = BilinearUpSampling2D((2,2))(input_layer)
    return output_layer

### Building the Model 

#### Encoder Block

In [3]:
#the 'filters' parameter defines the size or depth of the output layer
def encoder_block(input_layer, filters, strides):
    output_layer = separable_conv2d_batchnorm(input_layer, filters, strides)
    return output_layer

#### Decoder Block

In [4]:
def decoder_block(small_ip_layer, large_ip_layer, filters):
    
    #Upsample the small input layer
    upsample = bilinear_upsample(small_ip_layer)
    #Concatenate the upsampled and large input layers using layers.concatenate
    concatenate_upsample = layers.concatenate([upsample,large_ip_layer])
    #Add some number of separable convolution layers
    output_layer = separable_conv2d_batchnorm(concatenate_upsample,filters,1)
    output_layer = separable_conv2d_batchnorm(output_layer,filters,1)
    return output_layer

### FCN Architecture 

In [5]:
def fcn_model(inputs, num_classes):
    
    #Add Encoder Blocks. 
    filter = 96
    encoder_one = encoder_block(inputs,filter,2)
    encoder_two = encoder_block(encoder_one,filter *2, 2)
    encoder_three = encoder_block(encoder_two,filter*4,2)
    
    #Add 1x1 Convolution layer using conv2d_batchnorm().
    one_to_one_convolution = conv2d_batchnorm(encoder_three,filter*4,kernel_size=1,strides =1)
    
    #Add the same number of Decoder Blocks as the number of Encoder Blocks
    decoder_three = decoder_block(one_to_one_convolution,encoder_two,filter *4)
    decoder_two = decoder_block(decoder_three,encoder_one,filter *2)
    decoder_one = decoder_block(decoder_two,inputs,filter)
    
    #Returns the output layer of your model.
    return layers.Conv2D(num_classes, 3, activation='softmax', padding='same')(decoder_one)